In [137]:
from dotenv import load_dotenv; load_dotenv()
from datetime import datetime
import requests_cache
import os
import pandas_datareader.data as web
import pandas as pd

In [138]:
def get_prices(ticker, start, end):
    '''returns dataframe of adjusted close prices of specified ticker from start to end date'''
    start = datetime(start[0], start[1], start[2])
    end = datetime(end[0], end[1], end[2])
    api_key = os.getenv('ALPHAVANTAGE_API_KEY')
    session = requests_cache.CachedSession()

    prices = web.DataReader(ticker, 'av-daily-adjusted', start, end, api_key=api_key, session=session)
    prices = prices[['adjusted close']]
    prices.columns = [ticker]

    return prices

In [139]:
def get_all_prices(tickers, start, end):
    '''returns dataframe containing adjusted close prices of all specified tickers'''
    prices = []
    for ticker in tickers:
        prices.append(get_prices(ticker, start, end))
    return pd.concat(prices, axis=1)

In [140]:
def get_n_day_returns(prices, n=1):
    returns = (prices.shift(-n) / prices) - 1
    returns = returns[:-n]
    return returns

In [141]:
# testing stuff
tickers = ['AAPL', 'AMZN', 'GOOG', 'V', 'JPM']
start = (2017, 2, 9)
end = (2017, 2, 16)
prices = get_all_prices(tickers, start, end)
returns = get_n_day_returns(prices, 2)
prices,returns

(                AAPL    AMZN    GOOG        V      JPM
 2017-02-09  126.3876  821.36  809.56  83.6260  79.1495
 2017-02-10  126.1013  827.46  813.67  83.9387  78.9679
 2017-02-13  127.2180  836.53  819.24  84.4664  80.0117
 2017-02-14  128.8692  836.39  820.45  84.8670  81.2916
 2017-02-15  129.3369  842.70  818.98  85.7025  82.2265
 2017-02-16  129.1794  844.14  824.16  85.5752  82.1720,
                 AAPL      AMZN      GOOG         V       JPM
 2017-02-09  0.006570  0.018469  0.011957  0.010050  0.010893
 2017-02-10  0.021950  0.010792  0.008333  0.011059  0.029426
 2017-02-13  0.016656  0.007376 -0.000317  0.014634  0.027681
 2017-02-14  0.002407  0.009266  0.004522  0.008345  0.010830)